In [1]:
import pandas as pd

In [4]:
def compare_dois(csv1_df, csv2_df, start_year=2019):
    """
    Compare DOIs between two dataframes and identify unique entries in each.
    Filters second dataframe to include only entries from specified start year onwards.
    
    Parameters:
    csv1_df (pandas.DataFrame): First dataframe with DOI column
    csv2_df (pandas.DataFrame): Second dataframe with DOI column
    start_year (int): Starting year for filtering (default: 2019)
    
    Returns:
    pandas.DataFrame: A comparison dataframe showing which DOIs are unique to each source
    """
    # Filter CSV2 to include only entries from start_year onwards
    csv2_df_filtered = csv2_df[csv2_df['Year'] >= start_year].copy()
    
    # Create sets of DOIs from each dataframe
    dois_csv1 = set(csv1_df['DOI'].dropna())
    dois_csv2 = set(csv2_df_filtered['DOI'].dropna())
    
    # Find unique DOIs in each set
    unique_to_csv1 = dois_csv1 - dois_csv2
    unique_to_csv2 = dois_csv2 - dois_csv1
    
    # Create a list to store results
    comparison_results = []
    
    # Add entries unique to CSV1
    for doi in unique_to_csv1:
        paper = csv1_df[csv1_df['DOI'] == doi].iloc[0]
        comparison_results.append({
            'DOI': doi,
            'Source': 'CSV1 Only',
            'Title': paper['Title'],
            'Year': paper['Year'],
            'Authors': paper['Authors'] if 'Authors' in paper else None
        })
    
    # Add entries unique to CSV2
    for doi in unique_to_csv2:
        paper = csv2_df_filtered[csv2_df_filtered['DOI'] == doi].iloc[0]
        comparison_results.append({
            'DOI': doi,
            'Source': 'CSV2 Only',
            'Title': paper['Title'],
            'Year': paper['Year'],
            'Authors': paper['Author'] if 'Author' in paper else None
        })
    
    # Create comparison dataframe
    comparison_df = pd.DataFrame(comparison_results)
    
    # Add summary statistics
    print(f"Year filter applied: {start_year} onwards")
    print(f"Total entries in CSV1: {len(csv1_df)}")
    print(f"Total entries in CSV2 (after year filter): {len(csv2_df_filtered)}")
    print(f"Total unique DOIs in CSV1: {len(dois_csv1)}")
    print(f"Total unique DOIs in CSV2 (after year filter): {len(dois_csv2)}")
    print(f"DOIs unique to CSV1: {len(unique_to_csv1)}")
    print(f"DOIs unique to CSV2: {len(unique_to_csv2)}")
    print(f"DOIs common to both: {len(dois_csv1.intersection(dois_csv2))}")
    
    return comparison_df

In [ ]:
# Example usage:
csv1_df = pd.read_csv("csv1.csv")
csv2_df = pd.read_csv("csv2.csv")

In [6]:
# Ensure Year column is numeric
csv1_df['Year'] = pd.to_numeric(csv1_df['Year'], errors='coerce')
csv2_df['Year'] = pd.to_numeric(csv2_df['Year'], errors='coerce')

In [ ]:
comparison_df = compare_dois(csv1_df, csv2_df, start_year=2019)

In [8]:
# Add DOI link column
comparison_df['DOI_Link'] = comparison_df['DOI'].apply(lambda x: f'https://doi.org/{x}' if x else '')

In [10]:
# Select and reorder columns, excluding abstract
comparison_list_df = comparison_df[['Authors', 'Year', 'Title', 'Source', 'DOI_Link']]

In [ ]:
# Export to CSV
csv_path = 'comparison_list.csv'
comparison_list_df.to_csv(csv_path, index=False)
print(f"\nExported to {csv_path}")